In [ ]:
import matplotlib
matplotlib.use("PDF")
from pylab import *
from scipy import stats
thedir='/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole17/'
%cd $thedir
from scipy.special import lambertw, erf
from scipy.optimize import minimize, fmin_powell

# This one is used to determine the optimal QFe function


In [ ]:
def FOM(params):
    fom = 0.0
    [A, B, C, D] = params
    #[B, D] = params
    for i,ne in enumerate(Ne):
        qfe = QFe[i]
        ne_model = A * real(lambertw(exp(C * ( (B - qfe) - D * tanh(B - qfe)))))
        #print ne, ne_model
        if ne < 1.0:
            continue
        else:
            fom += abs((ne - ne_model)/ne)
    return fom


In [ ]:
def FOM(params):
    fom = 0.0
    [A, C] = params
    B = -1.0/.014993
    for i,ne in enumerate(Ne):
        qfe = QFe[i]
        ne_model = A/B(A + B * qfe) - A**2 / B * log(A + B * qfe) + C
        #print ne, ne_model
        if ne < 1.0:
            continue
        else:
            fom += abs((ne - ne_model)/ne)
    return fom


In [ ]:
clf()
QFe = []
Ne = []
lowQFe = []
lowNe = []
highQFe = []
highNe = []
file = open("data/run1/BF_256_9x9_QFe.dat")
lines = file.readlines()
del(lines[0]) # Strip the first line, which is labels
for line in lines:
    items = line.split()
    qfe = float(items[0])
    ne = float(items[1])
    #print "QFe = %f, Ne = %d"%(qfe, ne)
    if qfe > 15.0:
        continue
    QFe.append(qfe)
    Ne.append(ne)
    if ne > 0.0 and ne < 100.0:
        lowQFe.append(qfe)
        lowNe.append(ne)
    if ne > 100000.0:
        highQFe.append(qfe)
        highNe.append(ne)
print lowQFe, lowNe
lowslope, intercept, r_value, p_value, std_err = stats.linregress(lowQFe, log(lowNe))
print "Low slope sets the value of C. LowSlope = %.4f, r^2 = %.4f"%(lowslope, r_value)
highslope, intercept, r_value, p_value, std_err = stats.linregress(highQFe, highNe)
print "High slope sets the value of A. HighSlope = %.4f, r^2 = %.4f"%(highslope, r_value)
C = -lowslope
A = highslope / lowslope
B = 9.02
D = 0.727
param0 = [A, B, C, D]
#param0 = [B, D]
#print FOM(param0)

args = ()    
Result = fmin_powell(FOM, param0, args)
print A, C, Result
[A, B, C, D] = Result
#[B, D] = Result

for i,qfe in enumerate(QFe):
    ne = A * real(lambertw(exp(C * ( (B - qfe) - D * tanh(B - qfe)))))
    print "Qfe = %.4f, Ne meas = %.4f, Ne calc = %.4f"%(qfe, Ne[i], ne)
clf()
figure()
subplots_adjust(hspace = 1.0, wspace = 1.0)
xplot4 = linspace(2.0,15.0,500)
yplot4 = A * real(lambertw(exp(C * ( (B - xplot4) - D * tanh(B - xplot4)))))
yplot5 = -A * C * (1.0 - D / cosh(B-xplot4)**2) * real(lambertw(exp(C * ( (B - xplot4) - D * tanh(B - xplot4))))) / (real(lambertw(exp(C * ( (B - xplot4) - D * tanh(B - xplot4))))) + 1)
subplot(2,2,1)
scatter(QFe, -log10(Ne))
plot(xplot4, -log10(yplot4), color = 'magenta', ls = '--')
xlabel("Quasi-Fermi Level(V)")
ylabel("Number of Electrons")
ylim(-7, 0)
subplot(2,2,2)
scatter(QFe, 1/array(Ne))
plot(xplot4, yplot4, color = 'magenta', ls = '--')
xlabel("Quasi-Fermi Level(V)")
ylabel("Number of Electrons")
ylim(0.0,0.1)
subplot(2,2,3)
plot(xplot4, yplot5, color = 'red', ls = '--')
xlabel("Quasi-Fermi Level(V)")
ylabel("Slope")
savefig("QFE_Lookup.pdf")

In [ ]:
ktq = .015
Q0 = 7.0
y0 = 100000.0
m = 80000.0
N0 = 10000000.0
clf()
figure()
xplot = linspace(4.0,12.0,500)
yplot = 1.0 / (N0 * exp(-(Q0 - xplot)/ktq) + clip(1.0/(y0 - m * xplot), 0.0, 1.0E7))
#plot(xplot, log10(yplot))
#ylim(-1.0, 6.0)
plot(xplot, yplot)
ylim(0.0, 200000.0)
savefig("Temp.pdf")